# Structure search with CrySPY & PFP

結晶構造探索を行うツールであるCrySPYをMatlantis上のPFP力場を用いて動かす方法。

## 事前セットアップ

以下２つのライブラリを用意します。

**CrySPY**
 - https://github.com/Tomoki-YAMASHITA/CrySPY

執筆時点の最新版はv0.10.3: https://github.com/Tomoki-YAMASHITA/CrySPY/releases/tag/v0.10.3

**combo: cryspyの中で使われているライブラリ**

 - https://github.com/Tomoki-YAMASHITA/combo3
　これを clone & install (pip install combo してしまうと他の物が入ってしまうので注意してください。）


**LAMMPS: matlantis-lammpsを利用**

 - Experimental packageからコピーし、ipynbに従ってインストール。

In [ ]:
# Download CrySPY v0.10.3
!git clone https://github.com/Tomoki-YAMASHITA/CrySPY.git -b v0.10.3

In [ ]:
!pip install cython

In [ ]:
# Download combo3, and install.
!git clone https://github.com/Tomoki-YAMASHITA/combo3.git
!pip install -e ./combo3

 - pyxtal
   - docs: https://pyxtal.readthedocs.io/en/latest/
   - github: https://github.com/qzhu2017/PyXtal

CrySPYが内部で利用をしている、空間群の対称性を考慮した操作を行うためのライブラリ。

以下のコマンドでインストールできます。

In [ ]:
!pip install pyxtal

**その他パッケージのインストール**

CrySPYの実行は、Terminal上で行うため、Terminalを開き以下のコマンドでいくつかの追加パッケージをインストールしてください。
```
# python=3.7のActivate
source ~/.local/bin/activate

# その他パッケージのインストール
pip install importlib_metadata pyparsing packaging python-dateutil six typing_extensions
```

## 公式のLAMMPS Exampleの実行

CrySPYの実行は、python programを実行していく形となり、Terminal上で行うことを推奨します。

以下のようなコードをTerminal上で実行することで、CrySPY packageに事前に含まれているLAMMPS連携Exampleを実行することができます。

```
# python=3.7のActivate
source ~/.local/bin/activate

# LAMMPS Exampleのディレクトリへ移動 (ディレクトリ位置はユーザーごとに異なるので注意)
cd CrySPY/example/LAMMPS_bash_Si16_lj_RS

# CrySPYの実行
python ../../cryspy.py
```

※ 2回目以降の実行で、初期化してから実行したい場合は、出力として作成される `data`, `work` ディレクトリおよび `cryspy.out`, `cryspy.stat`, `lock_cryspy` のファイルのうち存在するものを削除してから実行してください。

上記のように、実行したい設定ファイル郡が格納されているディレクトリ以下に `cd` コマンドで移動後、その場所から`cryspy.py` を実行することでツールを動かすことができます。

`cryspy.py` はJobの進捗状況に応じて、

 - 1. 構造候補生成
 - 2.1 次の構造候補の計算に移る
 - 2.2 今行っている構造の計算が終了したかを確認
 - 2. すべての計算が終了したか判定

といったCheckを繰り返すため、 **定期的に `python ../../cryspy.py` 部分を呼び出す必要があります。**



例えば、上記のLAMMPS exampleの場合、正常に動作していると以下のようなログが出力されます。

**1回目の`python ../../cryspy.py`:**

```
2022/11/02 07:00:34
CrySPY 0.10.3
Start cryspy.py

Read input file, cryspy.in
Write input data in cryspy.out
Save input data in cryspy.stat

# --------- Generate initial structures
Compoisition [16] not compatible with symmetry 152 :spg = 152 retry.
Structure ID      0 was generated. Space group: 134 --> 131 P4_2/mmc
Structure ID      1 was generated. Space group:  45 -->  45 Iba2
Structure ID      2 was generated. Space group:  83 -->  83 P4/m
Structure ID      3 was generated. Space group: 134 --> 134 P4_2/nnm
Structure ID      4 was generated. Space group: 204 --> 204 Im-3
```

→候補構造を5つ生成しています。実際に生成される構造はRandomとなるため実行毎に異なります。


**2回目の`python ../../cryspy.py`:**

```
2022/11/02 07:09:57
CrySPY 0.10.3
Restart cryspy.py



# ---------- job status
ID      0: submit job, Stage 1
ID      1: submit job, Stage 1
```
ID=0, 1 の構造についてのみ、実際にエネルギー計算(構造最適化含む)が開始されます。

**3回目の`python ../../cryspy.py`:**

```
# ---------- job status
ID      0: Stage 1 Done!
    submitted job, ID      0 Stage 2
ID      1: Stage 1 Done!
    submitted job, ID      1 Stage 2
```

2回目の実行から、ある程度の時間を空けてから(構造最適化が終了したあとに)実行を行うと、

このような形で、実際にすべての計算が終了するまで、何度も`python ../../cryspy.py` を実行し確認する必要があります。

### watch コマンドによる実行


この操作は watch コマンドを利用することで、自動化することができます。

以下のコマンドは20秒ごとに、 `python ../../cryspy.py` を実行し続ける例です。

```
watch -n 20 python ../../cryspy.py
```


これをTerminalで実行し、待ち続けていると、CrySPYの構造探索が進み、最終的に以下のような出力が得られます。

```
2022/11/04 01:22:20
CrySPY 0.10.3
Restart cryspy.py



# ---------- job status
Done all structures!
```

## PFPを用いたExampleの実行


上記のLAMMPS Exampleでは、Si 結晶の構造探索をLJ potentialで行っていました。<br/>
ここでは、力場をPFPに変更して構造探索を行ってみましょう。

`LAMMPS_bash_Si16_lj_RS_PFP` ディレクトリにPFPを使うように設定ファイルを書き換えたファイルが存在します。
具体的には`calc_in/in.si_1` および `calc_in/in.si_2` がLAMMPSの入力設定ファイルになりますが、これらを以下のようにPFP力場を使うように書き換えています。

```
pair_style pfp_api v3.0.0
pair_coeff * * species Si
```

Terminalを起動し、 `LAMMPS_bash_Si16_lj_RS_PFP` ディレクトリに移動後、以下のコマンドを実行してみてください。

```
# python=3.7のActivate
source ~/.local/bin/activate

# PFP LAMMPS Exampleのディレクトリへ移動
cd LAMMPS_bash_Si16_lj_RS_PFP

# CrySPYの実行 (watchコマンドを使って定期実行するVersion)
watch -n 20 python ../CrySPY/cryspy.py
```

(実行には、`matlantis-lammps` のインストールが必要です。)

※ 2回目以降の実行で、初期化してから実行したい場合は、出力として作成される `data`, `work` ディレクトリおよび `cryspy.out`, `cryspy.stat`, `lock_cryspy` のファイルのうち存在するものを削除してから実行してください。

PFPを用いたExampleは上記のLJ potentialを使用した場合と比べ、構造最適化などに時間がかかるため、１つ１つのJobが終わるまでに時間がかかり、"ID      0: still queueing or running" のようなログがある程度長く表示され続けます (数分から10分程度)。
時間を待つと、全てのJobが終了します。


### PFPが正常に実行されているか確認する方法

まず注意点として、 **`cryspy.py` の実行は、Terminal上で行う必要があります。**<br/>
Jupyter LabのNotebook上のセルから `!python cryspy.py` と実行した場合には、"ID      0: still queueing or running" が表示されるが、PFPがBackgroundで実行されずにスタックしてしまうことが確認されています。

Terminal上で、"ID      1: still queueing or running" が表示され続けていて、正常に実行が進んでいるかわからないときは、`work/000001/log.struc`, `work/000001/out.si`, `work/000001/stage1_out.si` などを開き計算ログに進捗があるかを確認してください。("000001" は実行Jobの番号に応じて変更してください。)